In [1]:
import pandas as pd

import os
import warnings
warnings.filterwarnings('ignore')

Februari 2019

Vi funderade på en idé till vår trendsida och fastnade för [Stockholms Handelskammares siffror](https://www.chamber.se/rapporter/kompetenskrisen-hotar-foretagen.htm) på brist på kompetens som leder till ständiga vakanser på Stockholms arbetsmarknad. En siffra i rapporten är att man pga 14.000 ständiga vakanser missar 16 mdr i skatteintäkter per år i regionen. Men regionsuppdelningen var inte vad vi ville och de regionala siffrorna var dessvärre inte vad vi önskade.

Istället fastnade vi för arbetslöshetssiffror. Och då mer specifikt långtidsarbetslösa i relation till gruppen arbetslösa i stort. I Handelskammarens rapport kunde man läsa att långtidsarbetslösa idag är 40 procent av alla öppet arbetslösa, inskrivna hos arbetsförmedlingen i Stockholm. Vi bestämde oss därför kolla hur det ser ut i alla kommuner i denna relation. Hur ser andelen långtidsarbetslösa av totalt antal öppet arbetslösa ut per kommun? Hur stor andel av den arbetsföra befolkningen (16-64 år) är långtidsarbetslös?

---

**Resultatet av följande rek publicerades i tidningen 21 februari 2019, och finns att hitta [på webben här!](https://www.dagenssamhalle.se/nyhet/langtidsarbetslosa-storsta-utmaningen-26328)**

---

Siffrorna kommer från Arbetsförmedlingen där jag fick hjälp av en otroligt treliv handläggare. Det var han som skickade grunddatafilerna till mig!

Efter samtal med Arbetsförmedlingen så har jag fått reda på några viktiga grejer:

1. De som står som "summa" är alla inskrivna öppet arbetslösa. Dessa kan vara i utbildningar, praktik eller andra program som _INTE_ innebär subventionerade anställningar. Man jobbar med andra ord inte avlönat.
2. Långtidsarbetslös är när man varit inskriven hos arbetsförmedlingen som öppet arbetslös i 12 månader eller längre.
3. Eftersom antalet öppet arbetslösa kan variera så löpandes månad till månad kan en ranking av måttet långtidsarbetslösa som andel av totalen bli orättvist. Bättre är att jämföra med befolkningen i arbetsför ålder.

### Kommunerna

In [2]:
df = pd.read_csv('data/inskrivna_arbetslösa_kön_ålder_2006-2018.txt',sep=',',encoding='iso-8859-1')

In [3]:
df.shape

(215107, 9)

In [4]:
df.columns = df.columns.str.replace('\(','_').str.replace('\)','').str.lower()

In [5]:
df = df.groupby(['period','kommun_beskr']).sum().reset_index()

In [6]:
df.head()

,period,kommun_beskr,sum_insal,sum_ua06,sum_ua12,sum_ua24
0,2006-01,Ale,847,383,207,83
1,2006-01,Alingsås,1202,506,273,112
2,2006-01,Alvesta,474,197,94,31
3,2006-01,Aneby,188,62,26,7
4,2006-01,Arboga,589,267,159,59


Efter samtal med Johan Eklöf har jag fått reda på att inkluderat i siffrorna för de som varit utan arbete i längre än 6 månader (sum_ua06) är också de som varit utan arbete 12 och 24 månader. Dessa tre kolumner ska alltså tolkas som "utan arbete X månader eller längre" och kan därför inte summeras.

In [7]:
df = df.loc[df['period'].str.contains("-01")]

Vi fick kompletterande data från arbetsförmedlingen och därför lägger jag nu till siffror för januari månad, 2019 till grunddatat:

In [8]:
jan19 = pd.read_excel('data/jan19.xlsx')

In [9]:
jan19['period'] = "2019-01"

In [10]:
# Lägger till januari 2019:
df = pd.concat([df,jan19])

#### Befolkningen i arbetsför ålder

In [11]:
files = [x for x in os.listdir('data') if "BE01" in x]

In [12]:
tmp = pd.DataFrame()
for file in files:
    tmp = pd.concat([tmp,pd.read_csv(f"data/{file}",sep=',',encoding='iso-8859-1')]).reset_index(drop=True)

In [13]:
tmp.region = tmp.region.apply(lambda x: x[4:].strip())

In [14]:
tmp = tmp.groupby(['region','år']).sum().reset_index()
tmp.columns = ['kommun_beskr','period','befolkning']

In [15]:
tmp.dtypes

kommun_beskr    object
period           int64
befolkning       int64
dtype: object

In [16]:
df['period'] = df['period'].apply(lambda x: int(x[:4].strip()))

In [17]:
tmp['period'] = tmp['period']+1

In [18]:
df = df[df['period']>2009].merge(tmp,on=['kommun_beskr','period'],how='left')

In [19]:
if not os.path.isdir('res'):
    os.makedirs('res')

In [20]:
df['andel_arblös'] = ((df['sum_insal']/df['befolkning'])*100).round(1)
df['andel_långtid'] = ((df['sum_ua12']/df['befolkning'])*100).round(1)

In [21]:
df[df['period']==2019][['kommun_beskr','andel_långtid','andel_arblös']]\
    .sort_values('andel_långtid',ascending=False)\
    .reset_index(drop=True)\
    .to_excel('res/res1.xlsx',index=False)

In [22]:
df = df[df['period']==2019][['kommun_beskr','andel_långtid','andel_arblös']]\
    .sort_values('andel_långtid',ascending=False)\
    .reset_index(drop=True)

In [23]:
utrikes = pd.read_excel('data/jan19_utrikes.xlsx')

In [24]:
utrikes = utrikes.merge(tmp[tmp['period']==2019].drop('period',axis=1),on='kommun_beskr',how='left')[['kommun_beskr','sum_ua12','befolkning']]

In [25]:
utrikes['andel_långtid_utrik'] = ((utrikes['sum_ua12']/utrikes['befolkning'])*100).round(1)

Fruktansvärt många vändor här. Men såhär landade vi till slut. Vi bestämde oss för att strunta i årsjämförelser och enbart beskriva dagsläget i varje kommun. Men vi har också lagt till utrikesfödda i statistiken då det är en så pass viktig del för att begripa varför det ser ut som det gör. Det här har varit en pärs.

In [26]:
df.merge(utrikes[['kommun_beskr','andel_långtid_utrik']],
         on='kommun_beskr',
         how='left')[['kommun_beskr',
                      'andel_långtid_utrik',
                      'andel_långtid',
                      'andel_arblös']].to_excel('res/res2.xlsx',index=False)

### Riket

In [27]:
df = pd.read_csv('data/inskrivna_arbetslösa_kön_ålder_2006-2018.txt',sep=',',encoding='iso-8859-1')

In [28]:
df.columns = df.columns.str.replace('\(','_').str.replace('\)','').str.lower()

In [29]:
df = df.groupby(['period','kommun_beskr']).sum().reset_index()

In [30]:
jan19.head()

,kommun_beskr,sum_insal,sum_ua06,sum_ua12,sum_ua24,period
0,Karlshamn,1463,846,638,349,2019-01
1,Karlskrona,2540,1396,1027,542,2019-01
2,Olofström,686,441,330,173,2019-01
3,Ronneby,1833,1197,944,622,2019-01
4,Sölvesborg,696,385,275,147,2019-01


In [31]:
df = pd.concat([df,jan19])

För att kunna göra jämförelser år till år vill vi endast kolla januari månad.

In [32]:
df = df.loc[df['period'].str.contains("-01")]

Summera alla inskrivna totalt och de som varit inskrivna 12 månader eller längre.

In [33]:
df = df.groupby('period').sum()[['sum_insal','sum_ua12']]

In [34]:
df.index = df.index.str.replace('-01','')

In [35]:
df = df.reset_index()

Döper om kolumnnamnen:

In [36]:
df.columns = ['År','Öppet arbetslösa','Långtidsarbetslösa (12 mån+)']

In [37]:
df.to_excel('res/riket.xlsx',index=False)